<br>

# Introdução


In [ ]:
import pprint
import shutil
import tarfile
import tempfile
from pathlib import Path

import geopandas as gpd
import pandas as pd
from paths import input_path, input_path_down, output_path_gpkg


<br>

## List Files

Lista todos os arquivos que foram feitos download.

São arquivos em formato .tar, por município!


In [ ]:
# Lista Downloads
list_tar_files = list(input_path_down.rglob('*.tar'))
list_tar_files = list(set(list_tar_files))
list_tar_files.sort()

# Results
print(f'N Pastas {len(list_tar_files)}')
pprint.pprint(list_tar_files[:5])


<br>

Após isso criei uma lista com todos os arquivos .shp que estão dentro dos arquivos .tar.


In [ ]:
list_shape_files = []

for tar_file in list_tar_files:
    # print(tar_file)
    # list_shp = []
    with tarfile.open(tar_file, 'r') as tar_obj:
        for member in tar_obj.getmembers():
            extension = Path(member.name).suffix
            if extension == '.shp':
                # print(member)
                name = member.get_info()['name']
                # print(name)
                tar_file_virtual = f'tar://{tar_file.as_posix()}/{name}'
                # print(tar_file_virtual)
                list_shape_files.append(tar_file_virtual)

# Results
print(f'São {len(list_shape_files)} shapefiles')
pprint.pprint(list_shape_files[:5])


<br>

Criei função que converte cada item da célula em uma diversidade de colunas


In [ ]:
def find_infos(row):
    """
    _summary_

    :param my_url: _description_
    :type my_url: _type_
    :return: _description_
    :rtype: _type_
    """
    my_url = row['url']

    url_path = Path(my_url.replace('tar://', ''))
    url_path_part = url_path.relative_to(input_path_down)
    parts = url_path_part.parts[0:-1]
    return (
        url_path,
        url_path.relative_to(input_path_down),
        url_path_part.parts[0],
        url_path_part.parts[1],
        url_path_part.parts[2],
        url_path_part.name,
    )


def find_file_type(name):
    return name.split('_', maxsplit=2)[2]


In [ ]:
# Create Dataframe
df = pd.DataFrame(list_shape_files, columns=['url'])

# Create Columns
df[
    ['path', 'path_tar', 'nome_tarfile', 'sub_1', 'sub_2', 'nome_shape']
] = df.apply(
    find_infos,
    result_type='expand',
    axis=1,
)
df['nome_tipo'] = df['nome_shape'].apply(find_file_type)

# Results
df.head()


In [ ]:
# Tipos das Subbacias
list_sub_2 = list(set(df['sub_2']))
list_sub_2


In [ ]:
# Tipos de Shapefiles
list_nome_tipo = list(set(df['nome_tipo']))
list_nome_tipo.sort()
list_nome_tipo


<br>

## Temporários

Como não quero extrair os arquivos .tar, criei centenas de arquivos temporários.

... para que depois eu pudesse "copiar" a estrutura dal lista de arquivos.


In [ ]:
# Clear TempFiles Folder
input_path_tempfiles = input_path / 'temp_files'
input_path_tempfiles.mkdir(exist_ok=True)
shutil.rmtree(input_path_tempfiles)
input_path_tempfiles.mkdir(exist_ok=True)


In [ ]:
df_temp = df[0:5]
df_temp = df
for index, row in df_temp.iterrows():
    with tempfile.NamedTemporaryFile(
        dir=input_path_tempfiles, delete=False
    ) as tmp:
        # print(tmp.name)
        tmp.close()

    # Parts
    parts = Path(row['path_tar']).parts[1:]
    my_path = parts[:-1]
    filename = parts[-1]

    # Path
    new_filepath = input_path_tempfiles / Path(*my_path)
    new_filepath.mkdir(exist_ok=True, parents=True)
    print(new_filepath)

    # Rename
    shutil.move(tmp.name, new_filepath / filename)


In [ ]:
list_nome_tipo


In [ ]:
# APP
list_files_app = list(input_path_tempfiles.rglob('*APP/*APP.shp'))
list_files_app_uso = list(input_path_tempfiles.rglob('*APP/*APP_USO.shp'))

# HIDROGRAFIA
list_files_hidro_simples = list(
    input_path_tempfiles.rglob('*HIDROGRAFIA/*RIOS_SIMPLES.shp')
)
list_files_hidro_duplas = list(
    input_path_tempfiles.rglob('*HIDROGRAFIA/*RIOS_DUPLOS*.shp')
)
list_files_hidro_nascentes = list(
    input_path_tempfiles.rglob('*HIDROGRAFIA/*NASCENTE*.shp')
)
list_files_hidro_massa = list(
    input_path_tempfiles.rglob('*HIDROGRAFIA/*MASSA*_DAGUA*.shp')
)

# USO
list_files_uso = list(input_path_tempfiles.rglob('*USO/*USO*.shp'))


<br>

## Teste de ler Gpd


In [ ]:
list_choose = list_files_hidro_massa
list_shps = [f.relative_to(input_path_tempfiles) for f in list_choose]
shps = list_shps[1]
shps.parts[0]


In [ ]:
tar_file = input_path_down / f'{shps.parts[0]}.tar' / shps
tar_file = f'tar://{tar_file.as_posix()}'
tar_file


In [ ]:
gdf = gpd.read_file(tar_file)
gdf.head()


<br>

## Read Data and Concat

- gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')


In [ ]:
dict_lists = {
    'app': list_files_app,
    'app_uso': list_files_app_uso,
    'hidro_simples': list_files_hidro_simples,
    'hidro_duplas': list_files_hidro_duplas,
    'hidro_nascentes': list_files_hidro_nascentes,
    'hidro_massa': list_files_hidro_massa,
    'uso': list_files_uso,
}


In [ ]:
for k, v in dict_lists.items():
     print(k)
     
     # Etapa 1: Avalia se é possível ver os arquivos shp dentro do TAR
     list_fix = []
     list_choose = list(v)
     for shp in list_choose:
          # Paths
          my_path = shp.relative_to(input_path_tempfiles)
          tar_file = input_path_down / f'{my_path.parts[0]}.tar' / my_path
          tar_file = f'tar://{tar_file.as_posix()}'
          list_fix.append(tar_file)
          try:
               gdf = gpd.read_file(tar_file)

          except Exception as e:
               print(tar_file)
               print(e)
               pass

     # Etapa 2: Junta tudo!
     list_shp = [gpd.read_file(shp).to_crs(epsg=4326) for shp in list_fix]
     gdf = gpd.GeoDataFrame(pd.concat(list_shp, ignore_index=True), crs=4326)
     
     # Etapa 3: Save
     gdf.to_file(
          output_path_gpkg / f'{k}.gpkg',
          layer=f'{k}',
          driver='GPKG',
          OVERWRITE=True
     )
     print(f'Fim do {k}')

# End
print('Fim Geral')
     

In [ ]:
shutil.rmtree(input_path_tempfiles)
